In [1]:
from importlib import reload

import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)


sys.path.append(parent_dir)

import src
reload(src)

from src.utils import load_MNIST_data
from src.model import Simple_Model
from src.train import train_model, tune_model
from src.inference import evaluate_overall, evaluate_classwise, confusion_matrix_plot, plot_roc_curves

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt



In [2]:
train_dloader, val_dloader, test_dloader = load_MNIST_data(batch_size = 128, num_workers=4) # try increase batch size to 128

In [3]:
hyperparam_space = {
    "lr": {"type": "float", "low": 1e-5, "high": 1e-2, "log": True},
    "weight_decay": {"type": "float", "low": 1e-5, "high": 1e-2, "log": True},
    "out_channels": {"type": "int", "low": 4, "high": 16},
    "kernel_size": {"type": "int", "low": 3, "high": 7}
}

In [4]:
device = torch.device("mps")

In [ ]:

tune_model(Simple_Model, train_dloader, val_dloader, model_name="Simple_CNN_v3", device=device, n_trials=50, hyperparam_space=hyperparam_space, epochs=3, project_name="Fashion-MNIST-CNN_v3")

Optimising:  22%|██▏       | 11/50 [05:23<18:55, 29.12s/it]